# <center> KIT Praktikum NN: L2-regularized Logistic Least Squares Regression </center>

</br>
On this exercise, you are going to apply what you learn from the `numpy` tutorial in the implementation of L2-regularized Logistic Least Squares Regression (LLSR). I will provide you the formula by now (you can do it yourself after the next lecture!!!), first you should use pens and papers to vectorize them. Then you implement the full of the classifier based on your vectorized version.

<img src="../Images/LogisticRegression.png" style="width:298px;height:275px">

</br>
L2-regularized Logistic Least Squares Regression is similar to the standard Logistic Regression: It is a binary classifier containing only one layer, mapping the input features to only one output using sigmoid function. The differents here are two things: 
* Instead of the _binary crossentropy error_ for the loss, it uses the _squared error_.
* It is applied the L2-regularization.

Note that we will do an SGD training for this exercise. More specifically:
* There are $m$ data instance on the training set, each has $n$ input features. 
* $x_{i}^{(j)}$ denotes the $i^{th}$ input feature of the $j^{th}$ data instance.
* $y^{(j)}$ denotes the binary label ($0$ or $1$) of the $j^{th}$ data instance.
* $w_{i}$ denotes the weight connecting the $i^{th}$ input feature to the output.
* $b$ is the bias of the Logistic Least Squares Regression.

So the steps of an unvectorized version are:
* The weights are initialized using Xavier Initialization, the bias can be initialized as 0.
* Train over 5 epochs, each epoch we do those steps:
  *  Loop over every data instance $x^{(j)}$:
     * Calculate the output of the LLSR: $o^{(j)} = \sigma(\sum_{i=1}^{n} w_ix_i^{(j)} + b)$
     * Calculate the cost: squared error $c^{(j)} = (y^{(j)} - o^{(j)})^2$
     * The final loss function is L2-regularized: $l^{(j)} = \frac{1}{2}c^{(j)} + \frac{\lambda}{2}\sum_{i=1}^{n}w_i^2$. 
     * Update the weights: 
         * Loop over every weight $w_i$ and update once at a time: $w_i = w_i - \eta((o^{(j)}-y^{(j)})o^{(j)}(1-o^{(j)})x_i^{(j)} + \lambda w_i)$
     * Update the bias: $b = b - \eta (o^{(j)}-y^{(j)})o^{(j)}(1-o^{(j)})$
  *  Calculate the total loss (of the epoch): $L = \frac{1}{m}\sum_{j=1}^{m}l^{(j)}$. Print it out. 


The guideline is to avoid explicit for-loops. _Hint_: We cannot make the epoch loop disappears, but all other loops can be replaced by vectorization.

First inport numpy and matplotlib:

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import math


We will use LLSR for the MNIST_SEVEN task: predict a $28\times 28$-pixel image of a handwritten digit whether it is a "7" or not. This is a binary classification task. I did the data reading for you. There is 5000 images, I split the first 4000 images for training, 500 images for tuning, 500 images for test. On this exercise we do not need to tune anything, so we'd leave the tuning (called the _dev set_) untouch. The first field is the label ("0"-"9") of the image, the rest are the grayscale value of each pixel. 

In [4]:
data = np.genfromtxt("../Data/mnist_seven.csv", delimiter=",", dtype="uint8")
train, dev, test = data[:4000], data[4000:4500], data[4500:]

In [5]:
def normalize(dataset):
    X = dataset[:, 1:] / 255.     # Normalize input features
    Y = (dataset[:, 0] == 7) * 1  # Convert labels from 0-9 to Is7 (1) or IsNot7(0)
    return X.T,Y.reshape(1, -1)

In [6]:
X_train, Y_train = normalize(train)
print(X_train.shape)
print(Y_train.shape)

X_test, Y_test = normalize(test)
print(X_test.shape)
print(Y_test.shape)

# shuffle the training data since we do SGD
# we shuffle outside the training 
# since we want to compare unvectorized and vectorized versions
# It doesn't affect to batch training later
np.random.seed(8888)     # Do not change those seedings to make our results comparable
np.random.shuffle(train) 


(784, 4000)
(1, 4000)
(784, 500)
(1, 500)


# Unvectorized Version of Stochastic Gradient Descent

First the unvectorized version of training:

In [7]:
def sigmoid(x):
    return 1. /(1 + np.exp(-x))

In [6]:
def train_unvectorized(X_train, Y_train, lr=0.2, lambdar=0.0001, epochs=5):
    
    n = X_train.shape[0]
    m = X_train.shape[1]
    
    # Xavier Initialization
    np.random.seed(1234)
    w = np.random.randn(n) * (np.sqrt(2. / (n + 1)))
    b = 0

    for epoch in range(epochs):
        L = 0
        for j in range(m):   # Loop over every training instance
            x = X_train[:, j]
            y = Y_train[:, j]
            
            # Forward pass
            o = sigmoid(np.dot(w,x) + b)
            
            # Calculate the loss
            L += 0.5 * (y - o)**2 + 0.5 * lambdar * np.dot(w, w)
            
            # Backward pass and update the weights/bias
            delta_w = (o - y) * o * (1 - o) * x + lambdar * w
            w = w - lr * delta_w
            
            delta_b = (o - y) * o * (1 - o)
            b = b - lr * delta_b
        
        # Accumulate the total loss and print it
        L /= m
        print("Error of the epoch {0}: {1}".format(epoch + 1, L))
    
    return w, b
        

And the (unvectorized) inference:

In [7]:
def test_unvectorized(X_test, Y_test, w, b):
    
    n_test = X_test.shape[0]
    m_test = X_test.shape[1]
    corrects = 0
    
    for j in range(m_test):
        x = X_train[:, j]
        y = Y_train[:, j]

        # Forward pass
        o = sigmoid(np.dot(w, x) + b)
        # Evaluate the outputs
        corrects +=  1 if (y == (o >= 0.5)) else 0
        
    print("Accuracy of our LLSR:" + str((corrects * 100.) / m_test) + "%")
    
    return corrects


Test on our test data. The accuracy should be better than 89.2%. This high score 89.2% is the baseline, achieved by do nothing rather than predicting all images are not a "seven" :p.

In [24]:
w, b = train_unvectorized(X_train, Y_train)
_ = test_unvectorized(X_test, Y_test, w, b)

Error of the epoch 1: [0.01153695]
Error of the epoch 2: [0.00842141]
Error of the epoch 3: [0.00776708]
Error of the epoch 4: [0.0074246]
Error of the epoch 5: [0.00713938]
Accuracy of our LLSR:99.2%


# Vectorized Version of Stochastic Gradient Descent

Now we move to the vectorized version of training and inference, just replace for-loops and total-sums by $np.dot()$,  $np.sum()$ and the numpy pair-wise operations (you should do the vectorization using pens and papers first).

In [35]:
def train_vectorized(X_train, Y_train, lr=0.2, lambdar=0.0001, epochs=5):
    
    n = X_train.shape[0]
    m = X_train.shape[1]
    
    # Xavier Initialization
    np.random.seed(1234)
    w = np.random.randn(n) * (np.sqrt(2. / (n + 1)))
    b = 0

    for epoch in range(epochs):
        L = 0
        for j in range(m):

            x = X_train[:, j]
            y = Y_train[:, j]
            
            # Forward pass
            o = sigmoid(np.dot(w,x) + b)
            
            # Calculate the loss
            L += 0.5 * (y - o)**2 + 0.5 * lambdar * np.dot(w, w)
            
            # Backward pass and update the weights/bias
            delta_w = (o - y) * o * (1 - o) * x + lambdar * w
            w = w - lr * delta_w
            
            delta_b = (o - y) * o * (1 - o)
            b = b - lr * delta_b
            
        L /= m
        print("Error of the epoch {0}: {1}".format(epoch + 1, L))
    return w, b

And the vectorized inference (short, clear and fast):

In [9]:
def test_vectorized(X_test, Y_test, w, b):
    
    m_test = X_test.shape[1]
    result = np.zeros((1, m_test))
    
    o = sigmoid(np.dot(w, X_test) + b)
    result = (o > 0.5)
    corrects = np.sum(result == Y_test)

    print("Accuracy of our LLSR:" + str((corrects * 100.) / m_test) + "%")
    
    return corrects


Those following runs should return exact the same outputs like the (unvectorized) training and inference before but in less than a second. The vectorization should be more effective (much faster) if this is not an one-layer logistic regression but a deep network.

In [13]:
w, b = train_vectorized(X_train, Y_train)
_ = test_vectorized(X_test, Y_test, w, b)

NameError: name 'train_vectorized' is not defined

# Vectorized Version of Batch Gradient Descent 

Here is the fully vectorized version, batch training (vectorizing over training instances). The formula (you might be able to derive them after the next lecture):

$$ z = w \cdot X + b $$

$$ o = \sigma(z) $$

$$ C = \frac{1}{2m}\sum_{j=1}^{m}(y^{(j)}-o^{(j)})^2 $$

$$ R = \frac{1}{2m}\sum_{i=1}^{n}w_i^2 $$

$$ L = C + \lambda R $$

$$ \frac{\partial C}{\partial z^{(j)}} = \frac{1}{m}(o^{(j)} - Y^{(j)}) * o^{(j)} * (1 - o^{(j)}) $$

$$ \frac{\partial z^{(j)}}{\partial w_i} = x_i $$

$$ \Rightarrow \frac{\partial C}{\partial w} = \frac{\partial C}{\partial z} \cdot X^T $$

$$ \frac{\partial R}{\partial w} = \frac{1}{m}w $$ 

$$ \Rightarrow \frac{\partial L}{\partial w} = \frac{\partial C}{\partial w} + \lambda\frac{\partial R}{\partial w} $$

$$ \frac{\partial z}{\partial b} = 1 $$

$$ \Rightarrow \frac{\partial L}{\partial b} = \frac{\partial C}{\partial b} = \sum_{j=1}^{m}(o^{(j)} - Y^{(j)}) * o^{(j)} * (1 - o^{(j)}) $$

$$ w = w - \eta * \frac{\partial L}{\partial w} $$

$$ b = b - \eta *  \frac{\partial L}{\partial b} $$

In [14]:
def train_batch(X_train, Y_train, lr=0.1, lambdar=0.0001, epochs=5):
    
    n = X_train.shape[0]
    m = X_train.shape[1]

    # Xavier Initialization
    np.random.seed(1234)
    w = np.random.randn(1, n) * (np.sqrt(2. / (n + 1)))
    b = 0

    for epoch in range(epochs):

        # Forward pass
        o = sigmoid(np.dot(w, X_train) + b)
        
        # Calculate the loss 
        L = np.sum((Y_train - o)**2) / (2 * m) + lambdar * np.dot(w,w.T) / (2*m)
        
        # Backward pass and update the weights/bias
        delta_w = np.dot((o - Y_train) * o * (1 - o)/m, X_train.T) + lambdar * w / m
        w -= lr * delta_w
        
        delta_b = np.sum((o - Y_train) * o * (1 - o))/m
        b -= lr * delta_b
        
        print("Error of the epoch {0}: {1}".format(epoch + 1, L))
        
    return w, b
        

Since it is a batch training and requires different hyperparameters, the result might not be comparable to the SGD trainings above. 

In [25]:
w_batch, b_batch = train_batch(X_train, Y_train, lr=0.2, lambdar=0.0001, epochs=1000)
_ = test_vectorized(X_test, Y_test, w_batch, b_batch)

Error of the epoch 1: [[0.16775273]]
Error of the epoch 2: [[0.08054239]]
Error of the epoch 3: [[0.05484309]]
Error of the epoch 4: [[0.04705552]]
Error of the epoch 5: [[0.04340094]]
Error of the epoch 6: [[0.04123045]]
Error of the epoch 7: [[0.03973984]]
Error of the epoch 8: [[0.03860919]]
Error of the epoch 9: [[0.03768837]]
Error of the epoch 10: [[0.0368987]]
Error of the epoch 11: [[0.03619561]]
Error of the epoch 12: [[0.03555243]]
Error of the epoch 13: [[0.03495251]]
Error of the epoch 14: [[0.03438519]]
Error of the epoch 15: [[0.03384352]]
Error of the epoch 16: [[0.0333229]]
Error of the epoch 17: [[0.03282029]]
Error of the epoch 18: [[0.03233369]]
Error of the epoch 19: [[0.03186175]]
Error of the epoch 20: [[0.0314036]]
Error of the epoch 21: [[0.03095865]]
Error of the epoch 22: [[0.03052652]]
Error of the epoch 23: [[0.0301069]]
Error of the epoch 24: [[0.02969957]]
Error of the epoch 25: [[0.02930431]]
Error of the epoch 26: [[0.02892091]]
Error of the epoch 27: [[

Error of the epoch 299: [[0.0119777]]
Error of the epoch 300: [[0.01196609]]
Error of the epoch 301: [[0.01195455]]
Error of the epoch 302: [[0.01194307]]
Error of the epoch 303: [[0.01193164]]
Error of the epoch 304: [[0.01192027]]
Error of the epoch 305: [[0.01190895]]
Error of the epoch 306: [[0.01189769]]
Error of the epoch 307: [[0.01188649]]
Error of the epoch 308: [[0.01187534]]
Error of the epoch 309: [[0.01186424]]
Error of the epoch 310: [[0.0118532]]
Error of the epoch 311: [[0.01184221]]
Error of the epoch 312: [[0.01183128]]
Error of the epoch 313: [[0.0118204]]
Error of the epoch 314: [[0.01180957]]
Error of the epoch 315: [[0.01179879]]
Error of the epoch 316: [[0.01178806]]
Error of the epoch 317: [[0.01177739]]
Error of the epoch 318: [[0.01176676]]
Error of the epoch 319: [[0.01175619]]
Error of the epoch 320: [[0.01174566]]
Error of the epoch 321: [[0.01173519]]
Error of the epoch 322: [[0.01172476]]
Error of the epoch 323: [[0.01171438]]
Error of the epoch 324: [[0.

Error of the epoch 582: [[0.00999992]]
Error of the epoch 583: [[0.00999557]]
Error of the epoch 584: [[0.00999124]]
Error of the epoch 585: [[0.00998691]]
Error of the epoch 586: [[0.0099826]]
Error of the epoch 587: [[0.00997829]]
Error of the epoch 588: [[0.009974]]
Error of the epoch 589: [[0.00996971]]
Error of the epoch 590: [[0.00996544]]
Error of the epoch 591: [[0.00996117]]
Error of the epoch 592: [[0.00995692]]
Error of the epoch 593: [[0.00995268]]
Error of the epoch 594: [[0.00994845]]
Error of the epoch 595: [[0.00994422]]
Error of the epoch 596: [[0.00994001]]
Error of the epoch 597: [[0.00993581]]
Error of the epoch 598: [[0.00993162]]
Error of the epoch 599: [[0.00992744]]
Error of the epoch 600: [[0.00992327]]
Error of the epoch 601: [[0.0099191]]
Error of the epoch 602: [[0.00991495]]
Error of the epoch 603: [[0.00991081]]
Error of the epoch 604: [[0.00990668]]
Error of the epoch 605: [[0.00990256]]
Error of the epoch 606: [[0.00989845]]
Error of the epoch 607: [[0.0

Error of the epoch 840: [[0.00913959]]
Error of the epoch 841: [[0.00913701]]
Error of the epoch 842: [[0.00913443]]
Error of the epoch 843: [[0.00913186]]
Error of the epoch 844: [[0.00912929]]
Error of the epoch 845: [[0.00912672]]
Error of the epoch 846: [[0.00912416]]
Error of the epoch 847: [[0.00912161]]
Error of the epoch 848: [[0.00911905]]
Error of the epoch 849: [[0.00911651]]
Error of the epoch 850: [[0.00911396]]
Error of the epoch 851: [[0.00911142]]
Error of the epoch 852: [[0.00910888]]
Error of the epoch 853: [[0.00910635]]
Error of the epoch 854: [[0.00910383]]
Error of the epoch 855: [[0.0091013]]
Error of the epoch 856: [[0.00909878]]
Error of the epoch 857: [[0.00909627]]
Error of the epoch 858: [[0.00909376]]
Error of the epoch 859: [[0.00909125]]
Error of the epoch 860: [[0.00908875]]
Error of the epoch 861: [[0.00908625]]
Error of the epoch 862: [[0.00908375]]
Error of the epoch 863: [[0.00908126]]
Error of the epoch 864: [[0.00907878]]
Error of the epoch 865: [[

One thing to compare: the speed. Try to run the same number of epochs (1000) with SGD, vectorized training, you can see it still takes a long time to run compared to the fully batch training.

In [39]:
w, b = train_vectorized(X_train, Y_train, epochs=1000)
_ = test_vectorized(X_test, Y_test, w, b)

Error of the epoch 1: [0.01153695]
Error of the epoch 2: [0.00842141]
Error of the epoch 3: [0.00776708]
Error of the epoch 4: [0.0074246]
Error of the epoch 5: [0.00713938]
Error of the epoch 6: [0.00687611]
Error of the epoch 7: [0.0066666]
Error of the epoch 8: [0.00652798]
Error of the epoch 9: [0.00642635]
Error of the epoch 10: [0.00634764]
Error of the epoch 11: [0.00628761]
Error of the epoch 12: [0.00624198]
Error of the epoch 13: [0.00620708]
Error of the epoch 14: [0.00618005]
Error of the epoch 15: [0.0061588]
Error of the epoch 16: [0.00614188]
Error of the epoch 17: [0.00612826]
Error of the epoch 18: [0.00611722]
Error of the epoch 19: [0.00610822]
Error of the epoch 20: [0.00610085]
Error of the epoch 21: [0.0060948]
Error of the epoch 22: [0.00608982]
Error of the epoch 23: [0.00608572]
Error of the epoch 24: [0.00608235]
Error of the epoch 25: [0.0060796]
Error of the epoch 26: [0.00607735]
Error of the epoch 27: [0.00607553]
Error of the epoch 28: [0.00607408]
Error 

Error of the epoch 229: [0.0060986]
Error of the epoch 230: [0.0060986]
Error of the epoch 231: [0.0060986]
Error of the epoch 232: [0.0060986]
Error of the epoch 233: [0.0060986]
Error of the epoch 234: [0.0060986]
Error of the epoch 235: [0.0060986]
Error of the epoch 236: [0.0060986]
Error of the epoch 237: [0.0060986]
Error of the epoch 238: [0.0060986]
Error of the epoch 239: [0.0060986]
Error of the epoch 240: [0.0060986]
Error of the epoch 241: [0.0060986]
Error of the epoch 242: [0.0060986]
Error of the epoch 243: [0.0060986]
Error of the epoch 244: [0.0060986]
Error of the epoch 245: [0.0060986]
Error of the epoch 246: [0.0060986]
Error of the epoch 247: [0.0060986]
Error of the epoch 248: [0.0060986]
Error of the epoch 249: [0.00609859]
Error of the epoch 250: [0.00609859]
Error of the epoch 251: [0.00609859]
Error of the epoch 252: [0.00609859]
Error of the epoch 253: [0.00609859]
Error of the epoch 254: [0.00609859]
Error of the epoch 255: [0.00609859]
Error of the epoch 25

Error of the epoch 451: [0.00609859]
Error of the epoch 452: [0.00609859]
Error of the epoch 453: [0.00609859]
Error of the epoch 454: [0.00609859]
Error of the epoch 455: [0.00609859]
Error of the epoch 456: [0.00609859]
Error of the epoch 457: [0.00609859]
Error of the epoch 458: [0.00609859]
Error of the epoch 459: [0.00609859]
Error of the epoch 460: [0.00609859]
Error of the epoch 461: [0.00609859]
Error of the epoch 462: [0.00609859]
Error of the epoch 463: [0.00609859]
Error of the epoch 464: [0.00609859]
Error of the epoch 465: [0.00609859]
Error of the epoch 466: [0.00609859]
Error of the epoch 467: [0.00609859]
Error of the epoch 468: [0.00609859]
Error of the epoch 469: [0.00609859]
Error of the epoch 470: [0.00609859]
Error of the epoch 471: [0.00609859]
Error of the epoch 472: [0.00609859]
Error of the epoch 473: [0.00609859]
Error of the epoch 474: [0.00609859]
Error of the epoch 475: [0.00609859]
Error of the epoch 476: [0.00609859]
Error of the epoch 477: [0.00609859]
E

Error of the epoch 674: [0.00609859]
Error of the epoch 675: [0.00609859]
Error of the epoch 676: [0.00609859]
Error of the epoch 677: [0.00609859]
Error of the epoch 678: [0.00609859]
Error of the epoch 679: [0.00609859]
Error of the epoch 680: [0.00609859]
Error of the epoch 681: [0.00609859]
Error of the epoch 682: [0.00609859]
Error of the epoch 683: [0.00609859]
Error of the epoch 684: [0.00609859]
Error of the epoch 685: [0.00609859]
Error of the epoch 686: [0.00609859]
Error of the epoch 687: [0.00609859]
Error of the epoch 688: [0.00609859]
Error of the epoch 689: [0.00609859]
Error of the epoch 690: [0.00609859]
Error of the epoch 691: [0.00609859]
Error of the epoch 692: [0.00609859]
Error of the epoch 693: [0.00609859]
Error of the epoch 694: [0.00609859]
Error of the epoch 695: [0.00609859]
Error of the epoch 696: [0.00609859]
Error of the epoch 697: [0.00609859]
Error of the epoch 698: [0.00609859]
Error of the epoch 699: [0.00609859]
Error of the epoch 700: [0.00609859]
E

Error of the epoch 896: [0.00609859]
Error of the epoch 897: [0.00609859]
Error of the epoch 898: [0.00609859]
Error of the epoch 899: [0.00609859]
Error of the epoch 900: [0.00609859]
Error of the epoch 901: [0.00609859]
Error of the epoch 902: [0.00609859]
Error of the epoch 903: [0.00609859]
Error of the epoch 904: [0.00609859]
Error of the epoch 905: [0.00609859]
Error of the epoch 906: [0.00609859]
Error of the epoch 907: [0.00609859]
Error of the epoch 908: [0.00609859]
Error of the epoch 909: [0.00609859]
Error of the epoch 910: [0.00609859]
Error of the epoch 911: [0.00609859]
Error of the epoch 912: [0.00609859]
Error of the epoch 913: [0.00609859]
Error of the epoch 914: [0.00609859]
Error of the epoch 915: [0.00609859]
Error of the epoch 916: [0.00609859]
Error of the epoch 917: [0.00609859]
Error of the epoch 918: [0.00609859]
Error of the epoch 919: [0.00609859]
Error of the epoch 920: [0.00609859]
Error of the epoch 921: [0.00609859]
Error of the epoch 922: [0.00609859]
E

# Vectorized Version of Minibatch Gradient Descent

Finally, we can do minibatch training, it is the same as batch training (the formula) but one iteration runs over a subset of the whole dataset at a time, and those subsets (minibatches) are shuffle before training:

In [14]:
def train_minibatch(X_train, Y_train, batch_size=256, lr=0.1, lambdar=0.0001, epochs=50):
    
    n = X_train.shape[0]
    
    # Xavier Initialization
    np.random.seed(1234)
    w = np.random.randn(1, n) * (np.sqrt(2. / (n + 1)))
    b = 0

    for epoch in range(epochs):
        
        # Split into minibatches 
        X_mb = np.array_split(X_train, batch_size, axis = 1)
        Y_mb = np.array_split(Y_train, batch_size, axis = 1)
        
        nMB = len(X_mb)
        
        # We shuffle the minibatches of X and Y in the same way
        np.random.seed(12345)
        shuffleIndex = np.random.permutation(range(nMB))
        
        # Now we can do the training, we cannot vectorize over different minibatches
        # They are like our "epochs"
        for i in range(nMB): # CODE HERE
            
            # Extract a minibatch to do training
            X_current = X_mb[shuffleIndex[i]]
            Y_current = Y_mb[shuffleIndex[i]]
            m = X_current.shape[1]

            # Forward pass
            o = sigmoid(np.dot(w, X_current) + b)

            # Calculate the loss 
            L = np.sum((Y_current - o)**2) / (2 * m) + lambdar * np.dot(w,w.T) / (2*m)

            # Backward pass and update the weights/bias
            delta_w = np.dot((o - Y_current) * o * (1 - o)/m, X_current.T) + lambdar * w / m
            w -= lr * delta_w

            delta_b = np.sum((o - Y_current) * o * (1 - o))/m
            b -= lr * delta_b

            print("Error of the iteration {0}: {1}".format(epoch * nMB + i + 1, L))

    return w, b

Minibatch Training for this LLSR is very sensitive to hyperparameter choosing. Should use with early stopping. Do not supprise if the accurary is bad. Shuffling the minibatch also takes time, so do not run this with large number of epochs.

In [18]:
# Do not run this for more than 100 epochs!!!!!!!!!
w_minibatch, b_minibatch = train_minibatch(X_train, Y_train, batch_size=512, lr=0.2, lambdar=0.001, epochs=30)
_ = test_vectorized(X_test, Y_test, w_minibatch, b_minibatch)

Error of the iteration 1: [[0.18817173]]
Error of the iteration 2: [[0.05341161]]
Error of the iteration 3: [[0.06471922]]
Error of the iteration 4: [[0.07796589]]
Error of the iteration 5: [[0.02335172]]
Error of the iteration 6: [[0.01294241]]
Error of the iteration 7: [[0.03923408]]
Error of the iteration 8: [[0.04923941]]
Error of the iteration 9: [[0.04140023]]
Error of the iteration 10: [[0.00911098]]
Error of the iteration 11: [[0.01682363]]
Error of the iteration 12: [[0.08265146]]
Error of the iteration 13: [[0.02916169]]
Error of the iteration 14: [[0.08505642]]
Error of the iteration 15: [[0.08253714]]
Error of the iteration 16: [[0.0275098]]
Error of the iteration 17: [[0.02232014]]
Error of the iteration 18: [[0.03339493]]
Error of the iteration 19: [[0.03208679]]
Error of the iteration 20: [[0.02335513]]
Error of the iteration 21: [[0.01086258]]
Error of the iteration 22: [[0.0058929]]
Error of the iteration 23: [[0.08413989]]
Error of the iteration 24: [[0.01598963]]
Err

Error of the iteration 443: [[0.00267429]]
Error of the iteration 444: [[0.0146731]]
Error of the iteration 445: [[0.02306909]]
Error of the iteration 446: [[0.05035094]]
Error of the iteration 447: [[0.00619073]]
Error of the iteration 448: [[0.01517677]]
Error of the iteration 449: [[0.07746768]]
Error of the iteration 450: [[0.00528828]]
Error of the iteration 451: [[0.00543114]]
Error of the iteration 452: [[0.00369624]]
Error of the iteration 453: [[0.00186688]]
Error of the iteration 454: [[0.00206256]]
Error of the iteration 455: [[0.08115444]]
Error of the iteration 456: [[0.0181259]]
Error of the iteration 457: [[0.00391855]]
Error of the iteration 458: [[0.00713473]]
Error of the iteration 459: [[0.01214735]]
Error of the iteration 460: [[0.0200212]]
Error of the iteration 461: [[0.00356492]]
Error of the iteration 462: [[0.00832736]]
Error of the iteration 463: [[0.00117509]]
Error of the iteration 464: [[0.06449532]]
Error of the iteration 465: [[0.01176268]]
Error of the i

Error of the iteration 876: [[0.0043947]]
Error of the iteration 877: [[0.00273562]]
Error of the iteration 878: [[0.0003984]]
Error of the iteration 879: [[0.00049178]]
Error of the iteration 880: [[0.0022635]]
Error of the iteration 881: [[0.03875864]]
Error of the iteration 882: [[0.00096313]]
Error of the iteration 883: [[0.00285026]]
Error of the iteration 884: [[0.00144278]]
Error of the iteration 885: [[0.00189848]]
Error of the iteration 886: [[0.03664384]]
Error of the iteration 887: [[0.0131071]]
Error of the iteration 888: [[0.00309445]]
Error of the iteration 889: [[0.00390503]]
Error of the iteration 890: [[0.00376448]]
Error of the iteration 891: [[0.0027615]]
Error of the iteration 892: [[0.00093409]]
Error of the iteration 893: [[0.00163992]]
Error of the iteration 894: [[0.00374791]]
Error of the iteration 895: [[0.00122175]]
Error of the iteration 896: [[0.00516779]]
Error of the iteration 897: [[0.06764128]]
Error of the iteration 898: [[0.00176731]]
Error of the ite

Error of the iteration 1295: [[0.00053285]]
Error of the iteration 1296: [[0.01411988]]
Error of the iteration 1297: [[0.00623354]]
Error of the iteration 1298: [[0.00068875]]
Error of the iteration 1299: [[0.00439834]]
Error of the iteration 1300: [[0.00244242]]
Error of the iteration 1301: [[0.04652165]]
Error of the iteration 1302: [[0.00878978]]
Error of the iteration 1303: [[0.00069579]]
Error of the iteration 1304: [[0.00050824]]
Error of the iteration 1305: [[0.00074956]]
Error of the iteration 1306: [[0.00267696]]
Error of the iteration 1307: [[0.00079554]]
Error of the iteration 1308: [[0.00039179]]
Error of the iteration 1309: [[0.0148825]]
Error of the iteration 1310: [[0.00152756]]
Error of the iteration 1311: [[0.0033989]]
Error of the iteration 1312: [[0.00409254]]
Error of the iteration 1313: [[0.01670924]]
Error of the iteration 1314: [[0.02969473]]
Error of the iteration 1315: [[0.01877771]]
Error of the iteration 1316: [[0.00325464]]
Error of the iteration 1317: [[0.0

Error of the iteration 1675: [[0.00069157]]
Error of the iteration 1676: [[0.00086613]]
Error of the iteration 1677: [[0.00068377]]
Error of the iteration 1678: [[0.0130409]]
Error of the iteration 1679: [[0.01153398]]
Error of the iteration 1680: [[0.00345423]]
Error of the iteration 1681: [[0.03861587]]
Error of the iteration 1682: [[0.00075584]]
Error of the iteration 1683: [[0.00551926]]
Error of the iteration 1684: [[0.00097808]]
Error of the iteration 1685: [[0.00313129]]
Error of the iteration 1686: [[0.00075529]]
Error of the iteration 1687: [[0.00207424]]
Error of the iteration 1688: [[0.00426237]]
Error of the iteration 1689: [[0.00094384]]
Error of the iteration 1690: [[0.00495643]]
Error of the iteration 1691: [[0.00144276]]
Error of the iteration 1692: [[0.02293239]]
Error of the iteration 1693: [[0.00064792]]
Error of the iteration 1694: [[0.00066451]]
Error of the iteration 1695: [[0.00998298]]
Error of the iteration 1696: [[0.00246935]]
Error of the iteration 1697: [[0.

...

Error of the iteration 13485: [[0.00158104]]
Error of the iteration 13486: [[0.00106545]]
Error of the iteration 13487: [[0.00357217]]
Error of the iteration 13488: [[0.00103309]]
Error of the iteration 13489: [[0.00125135]]
Error of the iteration 13490: [[0.00120947]]
Error of the iteration 13491: [[0.00300333]]
Error of the iteration 13492: [[0.00151119]]
Error of the iteration 13493: [[0.00101425]]
Error of the iteration 13494: [[0.00114439]]
Error of the iteration 13495: [[0.00114047]]
Error of the iteration 13496: [[0.00922281]]
Error of the iteration 13497: [[0.00105592]]
Error of the iteration 13498: [[0.00373285]]
Error of the iteration 13499: [[0.00165952]]
Error of the iteration 13500: [[0.00105857]]
Error of the iteration 13501: [[0.00472198]]
Error of the iteration 13502: [[0.01827554]]
Error of the iteration 13503: [[0.00107361]]
Error of the iteration 13504: [[0.00094945]]
Error of the iteration 13505: [[0.00097242]]
Error of the iteration 13506: [[0.00129196]]
Error of t

Error of the iteration 13897: [[0.00127757]]
Error of the iteration 13898: [[0.0011042]]
Error of the iteration 13899: [[0.06330017]]
Error of the iteration 13900: [[0.00214686]]
Error of the iteration 13901: [[0.0021939]]
Error of the iteration 13902: [[0.00180634]]
Error of the iteration 13903: [[0.00099939]]
Error of the iteration 13904: [[0.00110034]]
Error of the iteration 13905: [[0.00125806]]
Error of the iteration 13906: [[0.00743756]]
Error of the iteration 13907: [[0.00146073]]
Error of the iteration 13908: [[0.00175469]]
Error of the iteration 13909: [[0.00117387]]
Error of the iteration 13910: [[0.00111525]]
Error of the iteration 13911: [[0.00628582]]
Error of the iteration 13912: [[0.00149977]]
Error of the iteration 13913: [[0.00338079]]
Error of the iteration 13914: [[0.00142084]]
Error of the iteration 13915: [[0.00121255]]
Error of the iteration 13916: [[0.01276624]]
Error of the iteration 13917: [[0.06516872]]
Error of the iteration 13918: [[0.00792835]]
Error of the

Error of the iteration 14275: [[0.00120598]]
Error of the iteration 14276: [[0.00360425]]
Error of the iteration 14277: [[0.00107802]]
Error of the iteration 14278: [[0.00145354]]
Error of the iteration 14279: [[0.02587987]]
Error of the iteration 14280: [[0.00305065]]
Error of the iteration 14281: [[0.00124998]]
Error of the iteration 14282: [[0.00125545]]
Error of the iteration 14283: [[0.00148017]]
Error of the iteration 14284: [[0.00550245]]
Error of the iteration 14285: [[0.00313696]]
Error of the iteration 14286: [[0.00171579]]
Error of the iteration 14287: [[0.00098338]]
Error of the iteration 14288: [[0.06162064]]
Error of the iteration 14289: [[0.00628028]]
Error of the iteration 14290: [[0.01236089]]
Error of the iteration 14291: [[0.05506606]]
Error of the iteration 14292: [[0.00208203]]
Error of the iteration 14293: [[0.00229966]]
Error of the iteration 14294: [[0.00309223]]
Error of the iteration 14295: [[0.00098764]]
Error of the iteration 14296: [[0.00542495]]
Error of t

Error of the iteration 14661: [[0.0037144]]
Error of the iteration 14662: [[0.00147521]]
Error of the iteration 14663: [[0.00096928]]
Error of the iteration 14664: [[0.00103936]]
Error of the iteration 14665: [[0.0034582]]
Error of the iteration 14666: [[0.00117238]]
Error of the iteration 14667: [[0.00152222]]
Error of the iteration 14668: [[0.00096132]]
Error of the iteration 14669: [[0.00192864]]
Error of the iteration 14670: [[0.00099206]]
Error of the iteration 14671: [[0.00116502]]
Error of the iteration 14672: [[0.00098575]]
Error of the iteration 14673: [[0.00104658]]
Error of the iteration 14674: [[0.00095854]]
Error of the iteration 14675: [[0.03640732]]
Error of the iteration 14676: [[0.00284567]]
Error of the iteration 14677: [[0.00109843]]
Error of the iteration 14678: [[0.00241861]]
Error of the iteration 14679: [[0.00097076]]
Error of the iteration 14680: [[0.01004935]]
Error of the iteration 14681: [[0.00122359]]
Error of the iteration 14682: [[0.00209062]]
Error of the

Error of the iteration 15037: [[0.00475695]]
Error of the iteration 15038: [[0.01770905]]
Error of the iteration 15039: [[0.0011197]]
Error of the iteration 15040: [[0.00098818]]
Error of the iteration 15041: [[0.0010127]]
Error of the iteration 15042: [[0.00129516]]
Error of the iteration 15043: [[0.01335358]]
Error of the iteration 15044: [[0.06264601]]
Error of the iteration 15045: [[0.00156442]]
Error of the iteration 15046: [[0.02304999]]
Error of the iteration 15047: [[0.00101898]]
Error of the iteration 15048: [[0.01336508]]
Error of the iteration 15049: [[0.00117944]]
Error of the iteration 15050: [[0.00596079]]
Error of the iteration 15051: [[0.0161875]]
Error of the iteration 15052: [[0.00333068]]
Error of the iteration 15053: [[0.00668385]]
Error of the iteration 15054: [[0.001324]]
Error of the iteration 15055: [[0.00113749]]
Error of the iteration 15056: [[0.00264979]]
Error of the iteration 15057: [[0.00110628]]
Error of the iteration 15058: [[0.01348895]]
Error of the it